# Tune GPT2 to generate positive reviews
> Optimise GPT2 to produce positive IMDB movie reviews using a BERT sentiment classifier as a reward function.

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-lib/documentation-images/resolve/main/gpt2_bert_training.png' width='600'>
<p style="text-align: center;"> <b>Figure:</b> Experiment setup to tune GPT2. The yellow arrows are outside the scope of this notebook, but the trained models are available through Hugging Face. </p>
</div>


In this notebook we fine-tune GPT2 (small) to generate positive movie reviews based on the IMDB dataset. The model gets the start of a real review and is tasked to produce positive continuations. To reward positive continuations we use a BERT classifier to analyse the sentiment of the produced sentences and use the classifier's outputs as rewards signals for PPO training.

## Setup experiment

### Import dependencies

In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
%pip install transformers trl==0.11.3 wandb

In [53]:
import torch
from tqdm import tqdm
import pandas as pd

from torch.nn.utils.rnn import pad_sequence

tqdm.pandas()

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, AutoModelForCausalLMWithValueHead, PPOConfig
from trl.core import LengthSampler

### Configuration

In [54]:
config = PPOConfig(
    model_name="lvwerra/gpt2-imdb",
    learning_rate=1.41e-5,
)

sent_kwargs = {"top_k": None, "function_to_apply": "none", "batch_size": 16}

/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(


You can see that we load a GPT2 model called `gpt2_imdb`. This model was additionally fine-tuned on the IMDB dataset for 1 epoch with the huggingface [script](https://github.com/huggingface/transformers/blob/main/examples/legacy/run_language_modeling.py) (no special settings). The other parameters are mostly taken from the original paper ["Fine-Tuning Language Models from Human Preferences"](
https://huggingface.co/papers/1909.08593). This model as well as the BERT model is available in the Huggingface model zoo [here](https://huggingface.co/models). The following code should automatically download the models.

## Load data and models

### Load IMDB dataset
The IMDB dataset contains 50k movie review annotated with "positive"/"negative" feedback indicating the sentiment.  We load the IMDB dataset into a DataFrame and filter for comments that are at least 200 characters. Then we tokenize each text and cut it to random size with the `LengthSampler`.

In [55]:
def build_dataset(
    config,
    dataset_name="stanfordnlp/imdb",
    input_min_text_length=2,
    input_max_text_length=8,
):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    # load imdb with datasets
    ds = load_dataset(dataset_name, split="train")
    ds = ds.rename_columns({"text": "review"})
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [56]:
dataset = build_dataset(config)


def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

### Load pre-trained GPT2 language models

We load the GPT2 model with a value head and the tokenizer. We load the model twice; the first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This serves as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original language model.

In [8]:
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

tokenizer.pad_token = tokenizer.eos_token

NameError: name 'AutoModelForCausalLMWithValueHead' is not defined

### Initialize PPOTrainer
The `PPOTrainer` takes care of device placement and optimization later on:

In [45]:
ppo_trainer = PPOTrainer(
    config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator
)

/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(


### Load BERT classifier
We load a BERT classifier fine-tuned on the IMDB dataset.

In [46]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug
sentiment_pipe = pipeline(
    "sentiment-analysis", model="lvwerra/distilbert-imdb", device=device
)

Device set to use cuda:0


The model outputs are the logits for the negative and positive class. We will use the logits for positive class as a reward signal for the language model.

In [47]:
text = "this movie was really bad!!"
sentiment_pipe(text, **sent_kwargs)

[{'label': 'NEGATIVE', 'score': 2.335048198699951},
 {'label': 'POSITIVE', 'score': -2.726576089859009}]

In [48]:
text = "this movie was really good!!"
sentiment_pipe(text, **sent_kwargs)

[{'label': 'POSITIVE', 'score': 2.557039976119995},
 {'label': 'NEGATIVE', 'score': -2.294790029525757}]

### Generation settings
For the response generation we just use sampling and make sure top-k and nucleus sampling are turned off as well as a minimal length.

In [49]:
gen_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

## Optimize model

### Training loop

The training loop consists of the following main steps:
1. Get the query responses from the policy network (GPT-2)
2. Get sentiments for query/responses from BERT
3. Optimize policy with PPO using the (query, response, reward) triplet

**Training time**

This step takes **~2h** on a V100 GPU with the above specified settings.

In [64]:
# from torch.cuda.amp import autocast, GradScaler

In [63]:
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)

generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

for epoch, batch in enumerate(tqdm(ppo_trainer.dataloader)):
    # Get query tensors from batch
    query_tensors = batch["input_ids"]

    # Generate responses - API requires individual tensor processing
    with torch.no_grad():
        # Sample a consistent length for all responses in this batch
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len

        # Process queries and store responses as individual tensors in a list
        response_tensors = []

        for query in query_tensors:
            # Generate response (must process one at a time per API constraint)
            full_response = ppo_trainer.generate(query, **generation_kwargs)

            # Extract only the newly generated tokens
            if len(full_response) > len(query):
                response = full_response[len(query):]
            else:
                # Handle edge case where response isn't longer than query
                response = torch.tensor([tokenizer.pad_token_id],
                                      device=query.device,
                                      dtype=query.dtype)

            response_tensors.append(response)

    # Store the unmodified list of response tensors
    # DO NOT convert to a single batched tensor with pad_sequence!

    # For decoding, we need to handle individual tensors
    responses_text = [tokenizer.decode(r, skip_special_tokens=True) for r in response_tensors]
    batch["response"] = responses_text

    # Compute sentiment scores
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)

    # Extract positive sentiment scores
    positive_scores = [
        torch.tensor(max((item["score"] for item in output if item["label"] == "POSITIVE"), default=0.0))
        for output in pipe_outputs
    ]

    # Run PPO step - passing the LIST of tensors, not a batched tensor
    stats = ppo_trainer.step(query_tensors, response_tensors, positive_scores)
    ppo_trainer.log_stats(stats, batch, positive_scores)

 20%|█▉        | 38/194 [08:33<28:20, 10.90s/it]/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_trainer.py:1246: UserWarning: The average ratio of batch (60.34) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
 25%|██▍       | 48/194 [10:02<23:44,  9.75s/it]/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_trainer.py:1246: UserWarning: The average ratio of batch (18.49) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_trainer.py:1246: UserWarning: The average ratio of batch (78.91) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
 26%|██▋       | 51/194 [10:37<26:46, 11.24s/it]/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_trainer.py:1246: UserWarning: The average ratio of batch (10.33) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
 29%|██▉       | 57/194 [11:41<24:00, 10.51s/it]/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_trainer.py:1246: UserWarning: The av

### Training progress
If you are tracking the training progress with Weights&Biases you should see a plot similar to the one below. Check out the interactive sample report on wandb.ai: [link](https://wandb.ai/huggingface/trl/runs/w9l3110g).

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2_tuning_progress.png' width='800'>
<p style="text-align: center;"> <b>Figure:</b> Reward mean and distribution evolution during training. </p>
</div>

One can observe how the model starts to generate more positive outputs after a few optimisation steps.

> Note: Investigating the KL-divergence will probably show that at this point the model has not converged to the target KL-divergence, yet. To get there would require longer training or starting with a higher initial coefficient.

## Model inspection
Let's inspect some examples from the IMDB dataset. We can use `ref_model` to compare the tuned model `model` against the model before optimisation.

In [82]:
from torch.nn.utils.rnn import pad_sequence
import torch, pandas as pd


In [83]:
print(torch.__version__)

2.6.0+cu124


In [7]:
import torch, pandas as pd
from torch.nn.utils.rnn import pad_sequence

# ───────────────────────── 0.  setup ─────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ref_model.to(device).eval()
def strip_prompt(batch_out, prompt_lens):
    """Drop the prompt part, keep only newly-generated tokens."""
    return [out[l : l + gen_len]          # slice from end of prompt to end of generation
            for out, l in zip(batch_out, prompt_lens)]
def pos_score(text: str) -> float:
    # HF pipeline returns list[dict]
    out = sentiment_pipe(text, **sent_kwargs)[0]
    return max((d["score"] for d in out if d["label"] == "POSITIVE"), default=0.)

# ───────────────────────── 1.  sample mini-batch ─────────────
bs = 16
dataset.set_format("pandas")
df_batch  = dataset[:].sample(bs)

queries     = df_batch["query"].tolist()
query_ids   = [torch.tensor(ids, dtype=torch.long) for ids in df_batch["input_ids"]]
prompt_lens = [len(t) for t in query_ids]

padded = pad_sequence(
    query_ids, batch_first=True, padding_value=tokenizer.pad_token_id
).to(device)

# ───────────────────────── 2.  generate (AMP) ────────────────
gen_len           = output_length_sampler()          # one length for the batch
generation_kwargs = dict(max_new_tokens=gen_len, **gen_kwargs)

with torch.no_grad(), torch.autocast("cuda"):        # AMP, modern API
    ref_out   = ref_model.generate(padded, **generation_kwargs)
    policy_out = model.generate(padded, **generation_kwargs)

resp_ref   = strip_prompt(ref_out,   prompt_lens)    # list[Tensor]
resp_after = strip_prompt(policy_out, prompt_lens)

# ───────────────────────── 3.  decode & rewards ──────────────
resp_txt_ref   = tokenizer.batch_decode(resp_ref,   skip_special_tokens=True)
resp_txt_after = tokenizer.batch_decode(resp_after, skip_special_tokens=True)

with torch.no_grad():
    rewards_ref =  [torch.tensor(pos_score(q + r), device=device)
                    for q, r in zip(queries, resp_txt_ref)]
    rewards_after = [torch.tensor(pos_score(q + r), device=device)
                     for q, r in zip(queries, resp_txt_after)]

# ───────────────────────── 4.  PPO update ────────────────────
stats = ppo_trainer.step(query_ids, resp_after, rewards_after)
ppo_trainer.log_stats(
    stats,
    dict(query=queries,
         response_before=resp_txt_ref,
         response_after=resp_txt_after),
    rewards_after,
)

# ───────────────────────── 5.  (optional) dataframe ──────────
pd.DataFrame({
    "query":           queries,
    "response_before": resp_txt_ref,
    "reward_before":   [r.item() for r in rewards_ref],
    "response_after":  resp_txt_after,
    "reward_after":    [r.item() for r in rewards_after],
})

NameError: name 'ref_model' is not defined

Looking at the reward mean/median of the generated sequences we observe a significant difference.

In [ ]:
print("mean:")
display(df_results[["rewards (before)", "rewards (after)"]].mean())
print()
print("median:")
display(df_results[["rewards (before)", "rewards (after)"]].median())

## Save model
Finally, we save the model and push it to the Hugging Face for later usage.

In [ ]:
model.save_pretrained("gpt2-imdb-pos-v2", push_to_hub=True)
tokenizer.save_pretrained("gpt2-imdb-pos-v2", push_to_hub=True)